# Multi-variable Linear Regression

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.enable_eager_execution()
tf.__version__

/home/ssojux2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ssojux2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ssojux2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ssojux2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

'1.14.0'

# Hypothesis

## $$ H(x_1, x_2, x_3) = \underline{w_1 x_1 + w_2 x_2 + w_3 x_3} + b $$

## $$ = w_1 x_1 + w_2 x_2 + w_3 x_3 $$ 

## $$ =  \begin{pmatrix} w_{ 1 } & w_{ 2 } & w_{ 3 } \end{pmatrix}\cdot \begin{pmatrix} x_{ 1 } \\ x_{ 2 } \\ x_{ 3 } \end{pmatrix} $$

## $$ H(x_1, x_2, x_3) = WX $$ 



In [2]:
x_data = [
    [1., 0., 3., 0., 5.],
    [0., 2., 0., 4., 0.]
]
y_data  = [1, 2, 3, 4, 5]

W = tf.Variable(tf.random_normal([1, 2], -1.0, 1.0))
b = tf.Variable(tf.random_normal([1], -1.0, 1.0))

learning_rate = tf.Variable(0.001)

for i in range(100):
   with tf.GradientTape() as tape:
    hypothesis = tf.matmul(W, x_data) + b # (1, 2) * (2, 5) = (1, 5)
    cost = tf.reduce_mean(tf.square(hypothesis - y_data))

    W_grad, b_grad = tape.gradient(cost, [W, b])
    W.assign_sub(learning_rate * W_grad)
    b.assign_sub(learning_rate * b_grad)
    
    if i % 10 == 0:
        print("step: {:3} \t cost: {:5.4f} \t w[0][0]: {:5.4f} \t w[0][1]: {:5.4f} \t b: {:5.4f}".format(
            i, cost.numpy(), W.numpy()[0][0], W.numpy()[0][1], b.numpy()[0]))

step:   0 	 cost: 141.7312 	 w[0][0]: -2.5953 	 w[0][1]: -0.5794 	 b: -1.9842
step:  10 	 cost: 105.5219 	 w[0][0]: -2.0586 	 w[0][1]: -0.4137 	 b: -1.7898
step:  20 	 cost: 78.6503 	 w[0][0]: -1.5985 	 w[0][1]: -0.2650 	 b: -1.6208
step:  30 	 cost: 58.6946 	 w[0][0]: -1.2044 	 w[0][1]: -0.1314 	 b: -1.4739
step:  40 	 cost: 43.8632 	 w[0][0]: -0.8667 	 w[0][1]: -0.0114 	 b: -1.3460
step:  50 	 cost: 32.8307 	 w[0][0]: -0.5775 	 w[0][1]: 0.0967 	 b: -1.2346
step:  60 	 cost: 24.6159 	 w[0][0]: -0.3299 	 w[0][1]: 0.1939 	 b: -1.1376
step:  70 	 cost: 18.4925 	 w[0][0]: -0.1179 	 w[0][1]: 0.2816 	 b: -1.0529
step:  80 	 cost: 13.9225 	 w[0][0]: 0.0636 	 w[0][1]: 0.3606 	 b: -0.9789
step:  90 	 cost: 10.5070 	 w[0][0]: 0.2188 	 w[0][1]: 0.4319 	 b: -0.9142


## Test Score


x1 | x2 | x3 | Y
---- | ---- | ----| ----
96 | 91 | 99 | 194
88 | 85 | 82 | 181
78 | 77 | 73 | 177
67 | 66 | 61 | 164
55 | 51 | 53 | 157

# Hypothesis using matrix

$$ [5, 3] \cdot [?, ?] = [5, 1] $$

$$ H(X) = XW + b $$


In [3]:
data = np.array([
    # X1,   X2,    X3,   y
    [ 96.,  91.,  99., 194. ],
    [ 88.,  85.,  82., 181. ],
    [ 78.,  77.,  73., 177. ],
    [ 67.,  66., 61., 164. ],
    [ 55.,  51.,  53., 157. ]
], dtype=np.float32)

# slice data
X = data[:, :-1]
y = data[:, [-1]]

W = tf.Variable(tf.random_normal([3, 1]))
b = tf.Variable(tf.random_normal([1]))

learning_rate = 0.000001

def predict(X):
    return tf.matmul(X, W) + b

print("epoch | cost")

n_epochs = 1000
for i in range(n_epochs):
    # tf.GradientTape() to record the gradient of the cost function
    with tf.GradientTape() as tape:
        cost = tf.reduce_mean((tf.square(predict(X) - y)))

    # Loss함수의 gradient를 계산한다.
    W_grad, b_grad = tape.gradient(cost, [W, b])

    # 파라미터 업데이트 (W and b)
    W.assign_sub(learning_rate * W_grad)
    b.assign_sub(learning_rate * b_grad)
    
    if i % 100 == 0:
        print("{:5} | {:10.4f}".format(i, cost.numpy()))

epoch | cost
    0 |  4807.5361
  100 |   457.6768
  200 |   453.6895
  300 |   453.2665
  400 |   452.8482
  500 |   452.4320
  600 |   452.0179
  700 |   451.6057
  800 |   451.1955
  900 |   450.7870


## 데이터를 기반으로예측해보자

In [4]:
def predict(X):
    return tf.matmul(X, W) + b # 위쪽에 선언되어 있다.

predict(X).numpy() # prediction, 예측값

array([[217.89566],
       [193.47142],
       [172.02069],
       [146.62013],
       [122.3805 ]], dtype=float32)

In [5]:
# 새로운 데이터에 대한 예측

predict([[ 89.,  95.,  92.],[ 84.,  92.,  85.]]).numpy() 

array([[203.40086],
       [190.92006]], dtype=float32)